In [1]:
# idea:
"""
read train files for frenchmed, from frenchpress
train embeddings using skipgram, cbow, fasttext
compare candidate words "patient, treatment, disease, solution, yellow" between frenchmed and frenchmed+frenchpress
    evaluate using spatial, gensim
"""

'\nread train files for frenchmed, from frenchpress\ntrain embeddings using skipgram, cbow, fasttext\ncompare candidate words "patient, treatment, disease, solution, yellow" between frenchmed and frenchmed+frenchpress\n    evaluate using spatial, gensim\n'

In [2]:
import gensim
import pandas as pd
import nltk

In [3]:
frenchmed_file = "./TP_ISD2020/QUAERO_FrenchMed/QUAERO_FrenchMed_traindev.ospl"

frenchpress_file = "./TP_ISD2020/QUAERO_FrenchPress/QUAERO_FrenchPress_traindev.ospl"

# read frenchmed
with open(frenchmed_file, "r") as f:
    frenchmed = f.readlines()

# read frenchpress
with open(frenchpress_file, "r") as f:
    frenchpress = f.readlines()


merged_corpora = frenchmed + frenchpress
merged_file = "merged_corpora.ospl"

#save merged corpora
with open(merged_file, "w") as f:
    f.writelines(merged_corpora)


In [4]:
# import fasttext
# #train fasttext skipgram model
# frenchmed_skipgram_model = fasttext.train_unsupervised(frenchmed_file, model='skipgram')
# merged_skipgram_model = fasttext.train_unsupervised(merged_file, model='skipgram')

# #train fasttext cbow model
# frenchmed_cbow_model = fasttext.train_unsupervised(frenchmed_file, model='cbow')
# merged_cbow_model = fasttext.train_unsupervised(merged_file, model='cbow')





In [5]:
nltk.download('punkt')
nltk.word_tokenize(frenchmed[0])

[nltk_data] Downloading package punkt to C:\Users\deus-
[nltk_data]     diabolus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['EMEA', '/', 'H', '/', 'C', '/', '551']

In [6]:
#tokenize the text
frenchmed_tokenized = [nltk.word_tokenize(sentence) for sentence in frenchmed]
merged_corpora_tokenized = [nltk.word_tokenize(sentence) for sentence in merged_corpora]

#train gensim skipgram model
frenchmed_gensim_skipgram_model = gensim.models.Word2Vec(frenchmed_tokenized, sg=1)
frenchmed_gensim_skipgram_model.train(frenchmed_file, epochs=10, total_words=frenchmed_gensim_skipgram_model.corpus_total_words)
merged_gensim_skipgram_model = gensim.models.Word2Vec(merged_corpora_tokenized, sg=1)
merged_gensim_skipgram_model.train(merged_file, epochs=10, total_words=merged_gensim_skipgram_model.corpus_total_words)


#train gensim cbow model
frenchmed_gensim_cbow_model = gensim.models.Word2Vec(frenchmed_tokenized, sg=0)
frenchmed_gensim_cbow_model.train(frenchmed_file, epochs=10, total_words=frenchmed_gensim_cbow_model.corpus_total_words)
merged_gensim_cbow_model = gensim.models.Word2Vec(merged_corpora_tokenized, sg=0) 
merged_gensim_cbow_model.train(merged_file, epochs=10, total_words=merged_gensim_cbow_model.corpus_total_words)


(150, 190)

In [7]:
models = {
    "frenchmed_gensim_skipgram_model": frenchmed_gensim_skipgram_model,
    "merged_gensim_skipgram_model": merged_gensim_skipgram_model,
    "frenchmed_gensim_cbow_model": frenchmed_gensim_cbow_model,
    "merged_gensim_cbow_model": merged_gensim_cbow_model
}

In [8]:
words = ["patient", "traitement", "maladie", "solution", "jaune"]

dictionary_similarities = {}

for word in words:
    dictionary_similarities[word] = {
        model_name: [elements[0] for elements in model.wv.most_similar(word)] if word in model.wv else "NaN" for model_name, model in models.items()
    }
# for word in words:
#     dictionary_similarities[word] = {
#         #yes I could've used a dictionary to get all the models, but I wanted to keep it not simple to piss off my teammate
#         # "frenchmed_skipgram": [elements[1] for elements in frenchmed_skipgram_model.get_nearest_neighbors(word)] if word in frenchmed_skipgram_model.get_words() else "NaN",
#         # "merged_skipgram": [elements[1] for elements in merged_skipgram_model.get_nearest_neighbors(word)] if word in merged_skipgram_model.get_words() else "NaN",
#         # "frenchmed_cbow": [elements[1] for elements in frenchmed_cbow_model.get_nearest_neighbors(word)] if word in frenchmed_cbow_model.get_words() else "NaN",
#         # "merged_cbow": [elements[1] for elements in merged_cbow_model.get_nearest_neighbors(word)] if word in merged_cbow_model.get_words() else "NaN",
#         # "frenchmed_gensim_skipgram": [elements[0] for elements in frenchmed_gensim_skipgram_model.wv.most_similar(word)] if word in frenchmed_gensim_skipgram_model.wv else "NaN",
#         # "merged_gensim_skipgram": [elements[0] for elements in merged_gensim_skipgram_model.wv.most_similar(word)] if word in merged_gensim_skipgram_model.wv else "NaN",
#         # "frenchmed_gensim_cbow": [elements[0] for elements in frenchmed_gensim_cbow_model.wv.most_similar(word)] if word in frenchmed_gensim_cbow_model.wv else "NaN",
#         # "merged_gensim_cbow": [elements[0] for elements in merged_gensim_cbow_model.wv.most_similar(word)] if word in merged_gensim_cbow_model.wv else "NaN"
        
#     }

df = pd.DataFrame(dictionary_similarities)

In [29]:
models

{'frenchmed_gensim_skipgram_model': <gensim.models.word2vec.Word2Vec at 0x217f14096d0>,
 'merged_gensim_skipgram_model': <gensim.models.word2vec.Word2Vec at 0x217f1498b50>,
 'frenchmed_gensim_cbow_model': <gensim.models.word2vec.Word2Vec at 0x217f1488090>,
 'merged_gensim_cbow_model': <gensim.models.word2vec.Word2Vec at 0x217f13718d0>}

In [24]:
# increase column width to allow for better visualization
pd.set_option('display.max_colwidth', 1000)
df

,patient,traitement,maladie,solution,jaune
frenchmed_gensim_skipgram_model,"[recommandé, pris, qui, avant, dès, possible, tout, qu, utiliser, cette]","[devra, Le, TYSABRI, médicaments, qui, lors, ce, association, qu, peut]","[infection, autres, un, adulte, lors, enfant, charge, médicaments, leur, efficacité]","[perfusion, injectable, flacon, contient, 300, mg, Chaque, 100, microgrammes, ml]","[diarrhée, goutte, Cmax, fièvre, HDPE, 18, seul, 14, démarche, conditionnés]"
merged_gensim_skipgram_model,"[infection, thérapie, LEMP, hypersensibilité, hépatique, précoce, Parkinson, Par, utilisée, allaiter]","[TYSABRI, ’, patients, diagnostic, natalizumab, Epivir, hépatique, insuffisance, VIH, ziconotide]","[Parkinson, VIH, LEMP, sévère, infection, étude, transmission, hépatique, lamivudine, présentant]","[injectable, perfusion, poudre, dose, Refludan, diluer, lévodopa, intraveineuse, ml, orale]","[maillot, Bastad, Ittifak, Kamch, triomphe, Thaïlandais, Abdesslam, Cooke, ATP, 380000]"
frenchmed_gensim_cbow_model,"[est, pendant, après, par, qui, plus, peut, que, Reconcile, La]","[en, des, dans, TYSABRI, risque, sur, les, par, Le, que]","[en, avec, risque, à, une, un, ou, et, Une, après]","[flacon, perfusion, contient, 100, pour, g, dose, 30, jour, 150]","[comprimés, 100, LE, dose, 5, contenant, Health, ;, orale, 30]"
merged_gensim_cbow_model,"[compositeur, couple, jugement, laboratoire, nourrisson, employer, sérum, natalizumab, salaire, électronique]","[biais, VIH, mécanisme, médicament, cancer, genou, financement, traité, foie, Polisario]","[augmentation, négociation, espèce, diminution, avancée, durée, dictature, centaine, prime, modification]","[dose, poudre, durée, augmentation, forme, carte, récompense, paire, perfusion, formule]","[Perez, Jiang, navire, col, Francisco, Printemps, prêtre, pancréas, veineuse, Québec]"


In [20]:
df['patient']['frenchmed_gensim_skipgram_model']

['recommandé',
 'pris',
 'qui',
 'avant',
 'dès',
 'possible',
 'tout',
 'qu',
 'utiliser',
 'cette']

In [27]:
# split the pd entries into pairs of original word [column] and the similarities found for each pair
# first, create a new df with columns "original word", "similar word", "model"
df_pairs = pd.DataFrame(columns=["original word", "similar word", "model"])
for word in words:
    for model in df.index:
        similar_words = df[word][model]
        if similar_words != "NaN":
            for similar_word in similar_words:
                # add very ineficiently the new entry to the df
                entry = {"original word": word, "similar word": similar_word, "model": model}
                df_pairs = pd.concat([df_pairs, pd.DataFrame(entry, index=[0])], ignore_index=True)

In [28]:
df_pairs

,original word,similar word,model
0,patient,recommandé,frenchmed_gensim_skipgram_model
1,patient,pris,frenchmed_gensim_skipgram_model
2,patient,qui,frenchmed_gensim_skipgram_model
3,patient,avant,frenchmed_gensim_skipgram_model
4,patient,dès,frenchmed_gensim_skipgram_model
...,...,...,...
195,jaune,Printemps,merged_gensim_cbow_model
196,jaune,prêtre,merged_gensim_cbow_model
197,jaune,pancréas,merged_gensim_cbow_model
198,jaune,veineuse,merged_gensim_cbow_model


In [ ]:
#tsne embeddings for 

In [19]:
words = list(frenchmed_gensim_skipgram_model.wv.index_to_key)

X = frenchmed_gensim_skipgram_model.wv[frenchmed_gensim_skipgram_model.wv.index_to_key]
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3)
X_tsne = tsne.fit_transform(X)

In [21]:
import plotly.express as px
df_2 = pd.DataFrame(X_tsne, columns=['x', 'y', 'z'])
df_2['word'] = words
fig = px.scatter_3d(df_2, x='x', y='y', z='z', text='word')
fig.show()